In [1]:
# pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3   ## To install transformers 4.50.0.dev0
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-12b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/malineni/envs/unsloth/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.15: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    NVIDIA GeForce RTX 4070. Num GPUs = 1. Max memory: 11.719 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [2]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [4]:
import os
def read_markdown_files(directory):
    markdown_contents = []
    text_contents = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".md"):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, "r", encoding="utf-8", errors='replace') as f:
                        content = f.read()
                        markdown_contents.append(content)
                except Exception as e:
                    print(f"Error reading file {file_path}: {e}")
            elif file.endswith(".txt"):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, "r", encoding="utf-8", errors='replace') as f:
                        content = f.read()
                        text_contents.append(content)
                except Exception as e:
                    print(f"Error reading file {file_path}: {e}")

    return markdown_contents, text_contents

In [5]:
docs_content, text_content = read_markdown_files(os.path.join(os.curdir, "../documentation"))
print(docs_content[2])

Can usability models help robots learn and improve interaction quality through task data and user feedback?Yes, usability models can indeed support learning in robots to enhance usability over time. In fact, considering usability during the design and development of robotic systems is of paramount importance.

## Use of Usability Models in Robots

Usability models can serve as a guide for how robots should interact with users, helping them to achieve their goals efficiently, effectively, and with a high degree of satisfaction. Over time, as the robot interacts more with its users, it can use these models to learn and adapt itself to better meet the user's needs.

## Task Data & User Feedback

### Task Data

Task data refers to the specific tasks that the robot is programmed to perform. This data can include information on the success or failure of tasks, the time taken to complete tasks, and other metrics that are relevant to the task's performance.

Robots can use this data to improve

In [6]:
from datasets import Dataset, load_dataset, concatenate_datasets
data1 = {"text": text_content}
datasets1 = Dataset.from_dict(data1)
data2 = {"text": docs_content}
datasets2 = Dataset.from_dict(data2)
datasets = concatenate_datasets([datasets1, datasets2])
EOS_TOKEN = tokenizer.eos_token

In [7]:
def formatting_prompts_func(examples):
    return { "text" : [example + EOS_TOKEN for example in examples["text"]] }
datasets = datasets.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 574/574 [00:00<00:00, 48563.40 examples/s]


In [8]:
for row in datasets[3:4]["text"]:
    print("=========================")
    print(row)

query: cut the apple into 4 equal pieces on the table using the sharp black knife


response: comprehensive cram action designator in lisp format is as below
(an action
    (type cutting)
    (object (an object
              (type apple)
              (name "standard-apple")
              (properties (size "medium")
                          (texture "smooth")
                          (color "red")))))
    (tool (a tool
            (type knife)
            (name "sharp-black-knife")
            (properties (sharpness "very-high")
                        (size "medium")
                        (material "steel")
                        (weight "light")
                        (color "black")
                        (edge "smooth")))))
    (location (a location
                (type table)
                (name "kitchen-table")
                (properties (material "wood")
                            (height 0.9)
                            (accessibility "high")
                       

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments
max_seq_length = 2048

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = datasets,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,
    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_ratio = 0.1,
        num_train_epochs = 5,
        learning_rate = 5e-5,
        embedding_learning_rate = 5e-6,
        max_steps=100,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.00,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=8): 100%|██████████| 574/574 [00:02<00:00, 192.52 examples/s]


In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 574 | Num Epochs = 3 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.682600
2,1.883200
3,1.771200
4,1.823200
5,1.746600
6,1.599800
7,1.807400
8,1.759300
9,1.720000
10,1.809200


In [11]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

prompts = [
    "what is the comprehensive action designator for the task- cut the apple. Give it in lisp format, no further explanation is needed",
    "what are the flanagan motion phases involved in the task- cut the apple. Give it in json format, no further explanation is needed",
    "what are the motion constraints involved in the task- cut the apple. Give it in json format, no further explanation is needed",
    "what are the framenet elements involved in the task- cut the apple. Give it in json format, no further explanation is needed",
    "what are the object and tools involved in the task- cut the apple. Give it in json format, no further explanation is needed"
]

inputs = tokenizer(
[
    prompts[1]
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 2048,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()

length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

<bos>what are the flanagan motion phases involved in the task- cut the apple. Give it in json
format, no further explanation is
needed<bos>what are the flanagan motion phases involved in the task- cut the apple. Give it in json format, no further explanation is needed.

```json
[
  {
    "phase": "initial_positioning",
    "description": "Positioning the 
hand and tool (knife) in front of the apple, ensuring proper alignment for cutting.",
    "duration": 
"0.1-0.2 seconds"
  },
  {
    "phase": "approach_and_grip",
    "description": "Moving the knife towards 
the apple and securely gripping it with the fingers.",
    "duration": "0.2-0.3 seconds"
  },
  {
    
"phase": "cutting_motion",
    "description": "Executing the cutting motion with the knife, applying force 
to slice through the apple.",
    "duration": "0.5-1.0 seconds"
  },
  {
    "phase": 
"separation_and_withdrawal",
    "description": "Separating the cut piece from the apple and smoothly withdrawing the knife.",

    "d